# PV181 RNG

This notebook contains python code for several tasks treated in this seminar. 

# PRNG 

# Task 1: determinism of PRNG

We will work with PRNG implemented in [random](https://docs.python.org/3/library/random.html) package. See first 4 methods (`seed, setstate, getstate, randbytes`) in the documentation. 
 1. Import **random** package.
 2. Generate (and print) 10 random bytes.  
 3. Print out bytes in hexadecimal form (use `.hex()` method of bytes). Execute cell 2x. </br> PRNG produced different results since  generation is not deterministic as it is seeded by time. 
 4. Use fixed `seed` and verify that generation is deterministic (generated bytes are always the same).

In [1]:
import random 


 5. Verify that seed determines internal state of the generator e.g. for `seed = 1` the internal state is `(3, (2147483648, ...)`? 
6. You can use seed to initialize state of PRNG or directly set the state. Setting PRNG to given state is helpful when you are working with the system that was randomly initialized or you do not know the seed. Generate same 10 bytes but use state (not seed) to initialize PRNG.     

 7. **Attack**: The generator produced 16 bytes that will be used as AES key. The first half of is `73a9bef499bbf4dca4f2`. Find the rest of the key. </br> **Hint**: user used small seed.

In [2]:
for seed in range(10):
    pass

 8. **Attack**: I generated yesterday after 3pm ten bytes b'\xcd\x8b\xa0\x0e\xed\xbf\xa4\xb0\xeb\x91' with time used as the seed `seed(int(time.time()))`. When exactly I generated the bytes and what was the seed? 

In [4]:
import random, time, datetime

rnd_bytes = b'\xcd\x8b\xa0\x0e\xed\xbf\xa4\xb0\xeb\x91'
t = int(time.time())
for s in range(t - 100000, t):
    if None == rnd_bytes:
        print("UTC:", datetime.datetime.utcfromtimestamp(s).isoformat())
        print("seed", s)

# LCG
Standard PRNG functions are very fast but also very weak. They leak internal state hence one generated value $r_i$ is enough to reseed the generator and regenerate next values $r_{i+1},r_{i+2},\cdots$. LCG is so weak (linear in the nature) that one can also go backwards (inverse generator) and find also the previous values $\cdots,r_{i-2},r_{i-1}$.
 * In python, PRNG [implemented](https://svn.python.org/projects/python/branches/release32-maint/Lib/random.py) in random module is [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister) with state formed by 625 32-bit integers. 
 * In other languages (C, Java, Rust) LCG is typically used. Internal state of LCG is **single** value (state) updated iterativelly as $$state = (state*a+c) \pmod m.$$ Overview of constants `a,c,m` used by the LCG for several languages can be found [here](https://en.wikipedia.org/wiki/Linear_congruential_generator).  
 <span style="color:red">In LCG, state is typically returned as generated random value!!</span>

# Task 2: common rand PRNG  
Following code was taken from [ANSI C standard](http://www.open-std.org/jtc1/sc22/wg14/www/docs/n1256.pdf#page=324) and simplified to other portable implementation (according to implementation of [rand()](https://code.woboq.org/userspace/glibc/stdlib/random_r.c.html#__random_r)) of seeding function `srand` and function for generation `rand()`. 

```
static unsigned long int next = 1;

void srand(unsigned int seed)
{
    next = seed;
}

int rand(void) 
{
    return next = (next * 1103515245 + 12345) & 0x7fffffff;
}
```
 1. Implement pythonic version of ANSI C PRNG - implement it as `class PRNG` below. </br> Use constants $a=1103515245, c=12345, m=2^{31}$ from [LCG wikipedia](https://en.wikipedia.org/wiki/Linear_congruential_generator).  
 2. Generate 10 values $12345, 1406932606, ...$ with LCG seeded by 0.  
 3. Generate 10 values but use different seed so the sequence will start with $1406932606, ...$.  

In [5]:
class PRNG:
    def __init__(self):
        self.srand(None) #replace None 


    def srand(self, seed):
        self.state = None #replace None 

    def rand(self):
        self.state = None #replace None 
        return None #replace None 

ansi_rand = PRNG()
ansi_rand.srand(0)
rnd_values = [ansi_rand.rand() for i in range (10)]
print(rnd_values)

[None, None, None, None, None, None, None, None, None, None]


 4. Every PRNG generates values in cycle i.e. generated sequence is periodic. Find the seed of the generator for which the generated sequence would be `[??, ??, 12345, 1406932606]`. Find previous two values (replaced by ??). </br>
 **Hint**:
 To revert this PRNG you can use constants $a^{-1}$ and $-(c*a^{-1})$ instead of constants $a$ and $c$.  </br>
 Compute $a^{-1}$ and $-(c*a^{-1})$ for ANSI C and generate (`[1406932606, 12345, ??, ??, seed]`).</br>
 In order to find backward LCG  it suffices to invert the update function: $$new\_state = old\_state*a+c \pmod m.$$   
 The inverse function can be computed as 
    $$
    \begin{align}
     old\_state &= (new\_state - c)/a \pmod m \\
                &= new\_state*(a^{-1}) - (c*a^{-1}) \pmod m \\
    \end{align}
    $$
  where $a^{-1} \pmod m$ can be computed using `*pow(a,-1,m)`. 

  
 

In [7]:
class PRNG:
    def __init__(self, a, c, m):
        self.a = a
        self.c = c
        self.m = m
        self.srand(1)

    def srand(self, seed):
        self.state = seed

    def rand(self):
        self.state = (self.state * self.a + self.c) % self.m
        return self.state
m = 1 #replace None 

a = 1 #replace 1 
c = 1 #replace 1 
ansi_rand_backward = PRNG(a, c, m)
ansi_rand_backward.srand(1)  #replace 1 
rnd_values = [ansi_rand_backward.rand() for i in range (7)]
print(rnd_values)

ansi_rand.srand(rnd_values[-1])
print([ansi_rand.rand() for i in range (6)])

[0, 0, 0, 0, 0, 0, 0]
[None, None, None, None, None, None]


  5. Use LCG.py script from **codes** folder and generate sequence of 10 numbers for java_util_random with `seed=1`. Following cmd will generate the sequence for you. Verify (google) that sequence is correct. </br>
  `python3 LCG.py -m 2**48 -a 25214903917 -c 11 -s 0 -l 16 -u 47 -n 10`

# Task 3: small state attack
 Generator `PRNG` below is uses cryptographic functions SHA1, SHA256 which are used to update the state and generate random value. As the functions are oneway (hard to invert), one can not recover the state from the generated value or go backwards and recover previous internal state of PRNG. Unfortunatelly, the problem of this PRNG is small state (1 Byte) seeded randomly from system source of randomness (see next section TRNG). 
 
 1. Explore the generator and see that it is seeded randomly and that the period is upperbounded  by $256=2^8$ for `state_size` of one byte (`state_size=1`). 
 
 - <span style="color:blue"> Interesting fact: Some values are not repeated. Why? </span>
   - There is a pre-period due to improperly designed generator. Hash functions are random functions not linear hence they do not produce cycles but repetition start from some point not from the begining like it is for LCG.

In [8]:
from cryptography.hazmat.primitives import hashes
import os

def SHA1(message: bytes):
    digest = hashes.Hash(hashes.SHA1())
    digest.update(message)
    return digest.finalize() 

def SHA256(message: bytes):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(message)
    return digest.finalize() 

class PRNG:
    def __init__(self, state_size = 1):
        self.state_size = state_size
        self.srand(os.urandom(16))

    def srand(self, seed):
        self.state = SHA256(seed)[:self.state_size]

    def rand_bytes(self, num_bytes=10):
        rnd = SHA256(self.state)[:num_bytes]
        self.state = SHA1(self.state)[:self.state_size]
        return bytes(rnd)

rng = PRNG()
sequence = [rng.rand_bytes(5).hex() for i in range(30)]
print(sequence)

['8d33f520a3', '7ace431cb6', '04b8d34e20', 'd4b0c0a4a8', '3f79bb7b43', '4b68ab3847', 'e4ff5e7d7a', 'bb7208bc9b', '01ba4719c8', '22adaf058a', '09fc96082d', 'dabd3aff76', 'd121100188', 'f8d20e598d', '2dbf9365a0', 'fb95aa98d6', 'd2e2adf717', '8de0b3c47f', 'dbc1b4c900', 'd121100188', 'f8d20e598d', '2dbf9365a0', 'fb95aa98d6', 'd2e2adf717', '8de0b3c47f', 'dbc1b4c900', 'd121100188', 'f8d20e598d', '2dbf9365a0', 'fb95aa98d6']


 2. **Attack**: Find all possible 16 byte blocks `rng` can produce.
    

In [9]:
all_keys = []
for i in range(1):
    pass #replace pass 
    K_candidate = rng.rand_bytes(16)
    all_keys.append(K_candidate)


 3. Previous generator was used to generate random keys `K1, K2`. Message b'arbitrarymessage' was encrypted to ciphertext `CT1` using  using `encrypt_ECB` and `K1`. Unknown message `PT2` was encrypted to `CT2`. 

In [10]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


def encrypt_ECB(key, msg):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    enc = cipher.encryptor()
    ct = enc.update(msg) + enc.finalize()
    return ct 

def decrypt_ECB(key, ct):
    cipher = Cipher(algorithms.AES(key),  modes.ECB())
    dec = cipher.decryptor()
    pt = dec.update(ct) + dec.finalize()
    return pt

rng = PRNG()
CT1 = encrypt_ECB(rng.rand_bytes(16), b'arbitrarymessage')
CT2 = b'\xb0\xc5\x1f5\x87,JH2\xd1\\8\xb0\xd4-Y'

 Find the key `K1`. Find the message `PT2`.

# TRNG

# Sources: dev/random, dev/urandom
These two files provide secure way to generate random bytes! 
Reading from dev/urandom can by done using following functions: 

In [11]:
import os
import secrets 
os.urandom(10)
print(secrets.token_bytes(10).hex())
print(os.getrandom(10).hex())

cff71a8f67e858876af1
767d16dabe6d5eaf0b41


Files **dev/random**, **dev/urandom** can be also opened as binary file for reading.  
Then you can read specified number of bytes e.g. 10. 

In [12]:
random_source = open("/dev/random", "rb")
random_source.read(10)

b'\xfd\x05\xd5\xc1k\x888\x1b\xc7\x00'

# Task 5: Testing correlation of bits
 1. Implement function `histogram(rnd_bytes, i, j)` that computes histogram of combination of bits (`i`-th and `j`-th bits of each byte). The function should return 4 frequencies for all bytes in `rnd_bytes` array. Frequencies will correspond to counts of how many bytes have `i`-th and `j`-th bit equal to combination 00,01,10 or 11.


In [13]:
import os
def histogram(rnd_bytes, i, j):
    hist = {0:0, (1 << i):0, (1 << j):0, (1 << i) + (1 << j): 0}
    mask = (1 << i) + (1 << j)
    for byte in rnd_bytes:
        hist[byte & mask] += 1
    return hist 



 2. Verify that for arbitrary params `i,j` and size of generated block the frequencies are roughly equal.  

In [14]:
rnd_bytes = os.urandom(1000)
histogram(rnd_bytes, 0, 1)

{0: 262, 1: 257, 2: 247, 3: 234}

 3. Generate random bytes using `ansi_rand` = generate integers, apply modulo, transform to bytes (use `bytes()`).
 4. Find params `i,j` where all the frequencies are exacly the same.  
 <span style="color:red"> Generator with such perfect results is also problematic! </span>   
 Can we predict some (next) bits with better probability than 50% ?